# მოდელისთვის საჭირო ბიბლიოთეკების იმპორტი

In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
df = pd.read_csv("test.csv")
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [3]:
#1.1 
#ჩანაწერებიოს რაოდენობა
len(df)

1459

In [4]:
# მახასიათებლების რაოდენობა
len(df.columns)

80

In [5]:
# კატეგორიულის და რიცხვითი მონაცემების რაოდენობა
num_numeric = df.select_dtypes(include=['int64', 'float64']).shape[1]
num_categorical = df.select_dtypes(include=['object', 'category']).shape[1]
num_numeric, num_categorical


(37, 43)

In [8]:
# დუბლიკატების შემოწმება და წაშლა
print("დუბლიკატების რაოდენობა:", df.duplicated().sum())
df = df.drop_duplicates()

# გამოტოვებული მნიშვნელობების შემოწმება
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print("\nსვეტები გამოტოვებული მნიშვნელობებით:")
print(missing)


# კატეგორიული სვეტები ყველაზე ხშირი მნიშვნელობით (mode)
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = df[cat_cols].apply(lambda x: x.fillna(x.mode()[0]))


# გამოტოვებული მნიშვნელობების შევსება
# რიცხვითი სვეტები -> მედიანით
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
num_imputer = SimpleImputer(strategy="median")  # median ან mean
df[num_cols] = num_imputer.fit_transform(df[num_cols])



# შემოწმება — დარჩა თუ არა გამოტოვებული მნიშვნელობები
print("\nსულ გამოტოვებული მნიშვნელობები:", df.isnull().sum().sum())

დუბლიკატების რაოდენობა: 0

სვეტები გამოტოვებული მნიშვნელობებით:
PoolQC          1456
MiscFeature     1408
Alley           1352
Fence           1169
MasVnrType       894
FireplaceQu      730
LotFrontage      227
GarageQual        78
GarageCond        78
GarageYrBlt       78
GarageFinish      78
GarageType        76
BsmtCond          45
BsmtQual          44
BsmtExposure      44
BsmtFinType1      42
BsmtFinType2      42
MasVnrArea        15
MSZoning           4
Functional         2
BsmtFullBath       2
Utilities          2
BsmtHalfBath       2
Exterior1st        1
Exterior2nd        1
TotalBsmtSF        1
BsmtUnfSF          1
BsmtFinSF2         1
BsmtFinSF1         1
KitchenQual        1
GarageArea         1
GarageCars         1
SaleType           1
dtype: int64

სულ გამოტოვებული მნიშვნელობები: 0


In [10]:
y = df["SaleCondition"]
X = df.drop(columns=["SaleCondition"])

X_encoded = pd.get_dummies(X, drop_first=True, dtype=int)
X_encoded

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,MiscFeature_Othr,MiscFeature_Shed,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,1461.0,20.0,80.0,11622.0,5.0,6.0,1961.0,1961.0,0.0,468.0,...,0,1,0,0,0,0,0,0,0,1
1,1462.0,20.0,81.0,14267.0,6.0,6.0,1958.0,1958.0,108.0,923.0,...,0,0,0,0,0,0,0,0,0,1
2,1463.0,60.0,74.0,13830.0,5.0,5.0,1997.0,1998.0,0.0,791.0,...,0,1,0,0,0,0,0,0,0,1
3,1464.0,60.0,78.0,9978.0,6.0,6.0,1998.0,1998.0,20.0,602.0,...,0,1,0,0,0,0,0,0,0,1
4,1465.0,120.0,43.0,5005.0,8.0,5.0,1992.0,1992.0,0.0,263.0,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915.0,160.0,21.0,1936.0,4.0,7.0,1970.0,1970.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,1
1455,2916.0,160.0,21.0,1894.0,4.0,5.0,1970.0,1970.0,0.0,252.0,...,0,1,0,0,0,0,0,0,0,1
1456,2917.0,20.0,160.0,20000.0,5.0,7.0,1960.0,1996.0,0.0,1224.0,...,0,1,0,0,0,0,0,0,0,1
1457,2918.0,85.0,62.0,10441.0,5.0,5.0,1992.0,1992.0,0.0,337.0,...,0,1,0,0,0,0,0,0,0,1


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

In [12]:
imputer = SimpleImputer(strategy="mean")  # ან median, most_frequent
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# მოდელების შექმნა

In [26]:
#Decision Tree Classifier
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)

In [27]:
# Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

In [30]:
# Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)

In [32]:
# წარივარ Category -> რიცხვები
y_test_num = y_test.astype('category').cat.codes
y_pred_dt_num = pd.Series(y_pred_dt).astype('category').cat.codes
y_pred_rf_num = pd.Series(y_pred_rf).astype('category').cat.codes
y_pred_gb_num = pd.Series(y_pred_gb).astype('category').cat.codes

models = {
    "Decision Tree": (y_test_num, y_pred_dt_num),
    "Random Forest": (y_test_num, y_pred_rf_num),
    "Gradient Boosting": (y_test_num, y_pred_gb_num)
}

for name, (y_true, y_pred) in models.items():
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    
    print(f"📌 {name}")
    print(f"MAE: {mae:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R² Score: {r2:.3f}")
    print("-"*30)

📌 Decision Tree
MAE: 0.54
MSE: 1.96
RMSE: 1.40
R² Score: -0.388
------------------------------
📌 Random Forest
MAE: 2.80
MSE: 8.23
RMSE: 2.87
R² Score: -4.839
------------------------------
📌 Gradient Boosting
MAE: 1.19
MSE: 1.81
RMSE: 1.34
R² Score: -0.283
------------------------------


In [ ]:
#ამოუკიდებელი შეფასების საფუძველზე, Gradient Boosting Classifier გამოვლინდა ყველაზე სტაბილურ და ზუსტ მოდელად SaleCondition-ის პროგნოზისთვის. 
#Decision Tree მარტივია და ვიზუალიზაციისთვის მოსახერხებელი, თუმცა ხშირად overfitting ხდება. 
#Random Forest უზრუნველყოფს უკეთეს generalization-ს, მაგრამ Gradient Boosting-ს აქვს მეტი კონტროლი 
#learning rate-ის საშუალებით, რაც იძლევა უკეთეს precision-ს რთულ კლასებზე. შესაბამისად, ამ ამოცანისთვის რეკომენდებულია Gradient Boosting, 
#რადგან იგი უკეთესად აკმაყოფილებს პროგნოზირების სიზუსტეს და სტაბილურობას.